In [898]:
import math
import pandas as pd

In [899]:
#constants
c = math.pi/180 #conversion factor
g = 9.81 #gravity

#initial conditions

#joint acceleration of ankle throughout entire movement
x_1 = 0
y_1 = 0

#angular velocity segment 1 during 1st interval
w_1 = 0
w_2 = 0
w_3 = 0
w_lb1 = 0
w_a = 0

#segment dimensions
L_1 = 0.48
L_2 = 0.44
L_3 = 0.55
L_lb1 = 0.12
L_a = 0.69

#masses
M = 180 #total
M_b = 300 #bar

M_1 = M*0.043
M_2 = M*0.11
M_3 = M*0.53
M_3 += M*0.07 #adding head to the torso "simplification"
M_b1 = 0
M_a = M*0.049

#proximal distances for each segment
P_1 = 0.4459
P_2 = 0.4095
P_3 = 0.4486
P_lb1 = 0.5
P_a = 0.5280

#moment of inertia for each segment
I_1 = 1/12*M_1*L_1**2
I_2 = 1/12*M_2*L_2**2
I_3 = 1/12*M_3*L_3**2
I_lb1 = 1/12*M_b1*L_lb1**2
I_a = 1/12*M_a*L_a**2

In [900]:
#to find angular accelerations and velocities for each segment
class angular_acceleration_velocity:

    #angular acceleration
    def a_i(O__i, O_i, t_i, w_i):
        return(2*(O__i-O_i)/(t_i**2)-w_i*t_i)

    #angular velocity
    def w_i(O__i, O_i, a_i):
        return(2*a_i*(O__i-O_i))

In [901]:
#to find linear accelerations of joints and segments
class accelerations:
    
    #general (joint)
    def x__i(x_i, L_i, a_i, w_i, theta): #x__i = x_(i+1)
        return(x_i-L_i*((a_i)*math.sin(theta)+w_i**2*math.cos(theta)))

    def y__i(y_i, L_i, a_i, w_i, theta):
        return(y_i+L_i*((a_i)*math.cos(theta)-w_i**2*math.sin(theta)))

    #general (center of mass)
    def x_c(x_i, P_i, L_i, a_i, w_i, theta):
        return(x_i-(1-P_i)*L_i*((a_i)*math.sin(theta)+w_i**2*math.cos(theta)))

    def y_c(y_i, P_i, L_i, a_i, w_i, theta):
        return(y_i+(1-P_i)*L_i*((a_i)*math.cos(theta)-w_i**2*math.sin(theta)))


In [902]:
#to find forces experienced by each segment
class forces:

    def Fx_12(Fx__i,M_i,x_c):
        return(Fx__i - M_i*x_c)

    def Fy_12(Fy__i,M_i,y_c):
        return(Fy__i + M_i*(y_c+g))

    def Fx_3(Fx_lb1,M_3,x_c): #torso+head
        return(Fx_lb1 - M_3*x_c)

    def Fy_3(Fy_lb1,M_3,y_c):
        return(Fy_lb1 + M_3*(y_c+g))

    def Fx_lb1(Fx_a):
        return(Fx_a)

    def Fy_lb1(Fy_a):
        return(Fy_a)

    def Fx_a(Fx_b, M_5, x_c):
        return(Fx_b - M_5*x_c)

    def Fy_a(Fy_b, M_5, y_c):
        return(Fy_b + M_5*(y_c+g))

    def Fx_b(M_b, x_b):
        return(-M_b*x_b)

    def Fy_b(M_b, y_b):
        return(M_b*(y_b+g))


In [903]:
#to find moments experienced by each segment
class moments:

    def m__i(m_i, a_i, Fx__i, Fy__i, Fx_i, Fy_i, theta):
        return(m_i-I_a*a_i-Fx__i*P_a*L_a*math.sin(theta)-Fy__i*P_a*L_a*math.cos(theta)-Fx_i*(1-P_a)*L_a*math.sin(theta)-Fy_i*(1-P_a)*L_a*math.cos(theta))

    def m_lb1(m_a, I_lb1, a_lb1, Fy_4, P_lb1, L_lb1, theta):
        return(-I_lb1*a_lb1+m_a-Fy_4*(1-P_lb1)*L_lb1*math.cos(theta))

In [904]:
#to find shear and compression forces for the spine

class shear_compression:

    def shear(Fx_i, Fy_i, M_i, x_i, y_i, theta):
        shear = abs(Fx_i*math.sin(theta)+Fy_i*math.cos(theta)-M_i*g*math.cos(theta)-(M_i*math.sqrt(x_i**2 + y_i**2))*math.cos(theta))
        return shear

    def compression(Fx_i, Fy_i, M_i, x_i, y_i, theta):
        compression = abs(Fx_i*math.cos(theta)-Fy_i*math.sin(theta)+M_i*g*math.sin(theta)+(M_i*math.sqrt(x_i**2 + y_i**2))*math.sin(theta))
        return compression

In [905]:
#interval 12
#inputs
#angle
O_1 = 80*c
O_2 = 209*c
O_3 = 40*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=180*c
O__3=43*c
O__4=180*c

#time
t_i = 0.5


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_4 = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_4)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_12 = pd.DataFrame(d, i)

#compression & shear output for interval 12
spine_c12 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s12 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [906]:
int_12

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-2872.655878,5172.094558,6880.757263
thigh,-0.537468,-0.027963,-2874.960937,5096.285085,5022.967308
torso,6.855079,1.753669,-2799.169891,4881.770094,1583.265763
shoulder,6.697184,1.921439,-2068.224235,3622.902953,2592.196142
arm,6.697184,1.921439,-2068.224235,3622.902953,2462.106377
bar,6.697184,1.921439,-2009.155075,3519.431662,0.000000


In [907]:
#interval 23
#inputs
#angle
O_1 = 88*c
O_2 = 180*c
O_3 = 43*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=173*c
O__3=47*c
O__4=180*c

#time
t_i = 0.4


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_4 = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_4)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_23 = pd.DataFrame(d, i)

#compression & shear output for interval 23
spine_c23 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s23 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [908]:
int_23

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-82.524812,5053.159143,3870.793841
thigh,0.059854,-0.002090,-82.268115,4977.238707,3934.654283
torso,0.491142,1.348782,-76.040435,4767.247829,594.142361
shoulder,0.141828,1.663802,-43.799204,3543.339548,2535.268326
arm,0.141828,1.663802,-43.799204,3543.339548,2408.035492
bar,0.141828,1.663802,-42.548284,3442.140614,0.000000


In [909]:
#interval 34
#inputs
#angle
O_1 = 88*c
O_2 = 173*c
O_3 = 47*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=163*c
O__3=57*c
O__4=180*c

#time
t_i = 1.4


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_4 = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_4)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_34 = pd.DataFrame(d, i)

#compression & shear output for interval 34
spine_c34 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s34 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [910]:
int_34

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-102.630039,4577.125430,3794.804307
thigh,0.000000,0.000000,-102.630039,4501.196030,3833.287476
torso,0.243845,0.505818,-99.779030,4301.044051,912.063204
shoulder,0.238795,0.509089,-73.744543,3186.740919,2280.121114
arm,0.238795,0.509089,-73.744543,3186.740919,2165.692882
bar,0.238795,0.509089,-71.638374,3095.726558,0.000000


In [911]:
#interval 45
#inputs
#angle
O_1 = 88*c
O_2 = 163*c
O_3 = 57*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=140*c
O__3=79*c
O__4=180*c

#time
t_i = 0.4


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_4 = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_4)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_45= pd.DataFrame(d, i)

#compression & shear output for interval 45
spine_c45 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s45 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [912]:
int_45

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-1661.856217,487.220934,2862.035566
thigh,0.000000,0.000000,-1661.856217,411.291534,1726.820490
torso,7.329984,-3.167293,-1576.154777,254.085210,834.477683
shoulder,3.314662,-9.993907,-1023.633775,-56.794091,-40.636314
arm,3.314662,-9.993907,-1023.633775,-56.794091,-38.596975
bar,3.314662,-9.993907,-994.398460,-55.172033,0.000000


In [913]:
#interval 56
#inputs
#angle
O_1 = 88*c
O_2 = 140*c
O_3 = 79*c
O_4 = 180*c

#angle'
O__1=88*c
O__2=125*c
O__3=84*c
O__4=180*c

#time
t_i = 0.4


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_a = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_a)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_56= pd.DataFrame(d, i)

#compression & shear output for interval 56
spine_c56 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s56 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [914]:
int_56

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-1563.803225,3850.935771,5913.692528
thigh,0.000000,0.000000,-1563.803225,3775.006371,4927.093251
torso,3.469486,-1.165016,-1523.238348,3594.389618,2597.302331
shoulder,3.678751,-1.189482,-1136.071898,2662.188320,1904.802414
arm,3.678751,-1.189482,-1136.071898,2662.188320,1809.209610
bar,3.678751,-1.189482,-1103.625314,2586.155353,0.000000


In [915]:
#interval 67
#inputs
#angle
O_1 = 88*c
O_2 = 125*c
O_3 = 84*c
O_4 = 180*c

#angle'
O__1=90*c
O__2=90*c
O__3=90*c
O__4=180*c

#time
t_i = 0.6


#accelerations (angular and linear)
#segment 1 & joint 2
a_1 = angular_acceleration_velocity.a_i(O__1, O_1, t_i, w_1)
w_1 = angular_acceleration_velocity.w_i(O__1, O_1, a_1)
x_1c = accelerations.x_c(x_1, P_1, L_1, a_1, w_1, O__1)
y_1c = accelerations.y_c(y_1, P_1, L_1, a_1, w_1, O__1)
x_2 = accelerations.x__i(x_1, L_1, a_1, w_1, O__1)
y_2 = accelerations.y__i(y_1, L_1, a_1, w_1, O__1)

#segment 2 & joint 3
a_2 = angular_acceleration_velocity.a_i(O__2, O_2, t_i, w_2)
w_2 = angular_acceleration_velocity.w_i(O__2, O_2, a_2)
x_2c = accelerations.x_c(x_2, P_2, L_2, a_2, w_2, O__2)
y_2c = accelerations.y_c(y_2, P_2, L_2, a_2, w_2, O__2)
x_3 = accelerations.x__i(x_2, L_2, a_2, w_2, O__2)
y_3 = accelerations.y__i(y_2, L_2, a_2, w_2, O__2)

#segment 3 & joint 4
a_3 = angular_acceleration_velocity.a_i(O__3, O_3, t_i, w_3)
w_3 = angular_acceleration_velocity.w_i(O__3, O_3, a_3)
x_3c = accelerations.x_c(x_3, P_3, L_3, a_3, w_3, O__3)
y_3c = accelerations.y_c(y_3, P_3, L_3, a_3, w_3, O__3)
x_lb1 = accelerations.x__i(x_3, L_3, a_3, w_3, O__3)
y_lb1 = accelerations.y__i(y_3, L_3, a_3, w_3, O__3)

#segment 4 & joint 5
a_lb1 = 0
w_lb1 = 0

#segment 5 & joint 6
a_a = angular_acceleration_velocity.a_i(O__4, O_4, t_i, w_a)
w_a = angular_acceleration_velocity.w_i(O__4, O_4, a_a)
x_lb1c = accelerations.x_c(x_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
y_lb1c = accelerations.y_c(y_lb1, P_lb1, L_lb1, a_4, w_a, O__4)
x_a = accelerations.x__i(x_lb1, L_lb1, a_4, w_a, O__4)
y_a = accelerations.y__i(y_lb1, L_lb1, a_4, w_a, O__4)

#the bar
x_b = x_a
y_b = y_a


#forces
Fy_b = forces.Fy_b(M_b, y_b)
Fx_b = forces.Fx_b(M_b, x_b)

Fy_a = forces.Fy_a(Fy_b,M_a,y_a)
Fx_a = forces.Fx_a(Fx_b,M_a,x_a)

Fx_lb1 = Fx_a 
Fy_lb1 = Fy_a

Fy_3 = forces.Fy_3(Fy_lb1,M_3,y_3c)
Fx_3 = forces.Fx_3(Fx_lb1,M_3,x_3c)

Fy_2 = forces.Fy_12(Fy_3,M_2,y_2c)
Fx_2 = forces.Fx_12(Fx_3,M_2,x_2c)

Fy_1 = forces.Fy_12(Fy_2,M_1,y_1c)
Fx_1 = forces.Fx_12(Fx_2,M_1,x_1c)


#moments
m_a = moments.m__i(0, a_4, Fx_b, Fy_b, Fx_a, Fy_a, O__4)

m_lb1 = moments.m_lb1(m_a, I_lb1, a_lb1, Fy_a, P_lb1, L_lb1, 180)

m_3 = moments.m__i(m_lb1, a_3, Fx_3, Fy_3, Fx_lb1, Fy_lb1, O__3)

m_2 = moments.m__i(m_3, a_2, Fx_3, Fy_3, Fx_2, Fy_2, O__2)

m_1 = moments.m__i(m_2, a_1, Fx_2, Fy_2, Fx_1, Fy_1, O__1)

#output
d = {'x_acc':[x_1, x_2, x_3, x_lb1, x_a, x_b], 'y_acc':[y_1, y_2, y_3, y_lb1, y_a, y_b], 'x_force':[Fx_1, Fx_2, Fx_3, Fx_lb1, Fx_a, Fx_b], 'y_force':[Fy_1, Fy_2, Fy_3, Fy_lb1, Fy_a, Fy_b], 'moment':[m_1, m_2, m_3, m_lb1, m_a, 0]}

i = ['shank', 'thigh', 'torso', 'shoulder', 'arm', 'bar']

int_67= pd.DataFrame(d, i)

#compression & shear output for interval 56
spine_c67 = shear_compression.compression(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

spine_s67 = shear_compression.shear(Fx_3, Fy_3, M_3, x_3, y_3, O__3)

In [916]:
int_67

,x_acc,y_acc,x_force,y_force,moment
shank,0.000000,0.000000,-765.466715,-2528.262644,110.989419
thigh,-0.093084,-0.000088,-765.865929,-2604.191667,-417.260195
torso,2.083065,-16.064889,-742.265681,-2610.599879,-938.840342
shoulder,1.741026,-16.074219,-537.663533,-1934.516236,-1384.151215
arm,1.741026,-16.074219,-537.663533,-1934.516236,-1314.687372
bar,1.741026,-16.074219,-522.307687,-1879.265821,0.000000


In [917]:
#spine shear and compression output
d = {'Interval 12':[spine_s12, spine_c12], 'Interval 23':[spine_s23, spine_c23], 'Interval 34':[spine_s34, spine_c34], 'Interval 45':[spine_s45, spine_c45], 'Interval 56':[spine_s56, spine_c56], 'Interval 67':[spine_s67, spine_c67]}

i = ['Shear', 'Compression']

shear_compression= pd.DataFrame(d, i)

In [918]:
shear_compression

,Interval 12,Interval 23,Interval 34,Interval 45,Interval 56,Interval 67
Shear,327.523320,2367.352015,1648.770991,1865.423049,1291.240137,742.265681
Compression,4132.802297,2650.170734,2722.086811,1336.389805,2287.145069,5419.612588
